# Extractor y Comparador de XLSX (Docencia) — **con `row_index`**

Este cuaderno:
1. **Carga** un archivo Excel (`.xlsx`) con **todas** sus hojas.
2. **Normaliza encabezados** (acentos → ASCII, `snake_case`) y unifica en una sola tabla (`master`).
3. Agrega como **identificador** la columna **`row_index`** (número de fila del `master`), sin hashes.
4. **Compara** una columna entre **dos** archivos `.xlsx` (valores solo en A, solo en B y comunes).

**Sugerencia:** Ejecuta las celdas de arriba hacia abajo. Edita las rutas en la celda de *Configuración*.


In [36]:
# (Opcional) Instala dependencias si las necesitas en tu entorno
# Quita el comentario si hace falta:
# %pip install -q pandas openpyxl
import pandas as pd
import re, unicodedata
from typing import Dict, List, Union


## Configuración
- Edita las rutas a tus archivos `.xlsx`.
- `columna_a_comparar` es el nombre **tal cual aparece** en tu Excel (el cuaderno lo normaliza y busca).


In [37]:
# === Configuración ===
excel_a = r"C:\Users\andra\Documents\8vo\Vice\automatizacion\ProyectoPreplanificacion\Comparacion2025A\data\2025A.xlsx"  # <-- cambia aquí
excel_b = r"C:\Users\andra\Documents\8vo\Vice\automatizacion\ProyectoPreplanificacion\Comparacion2025A\data\2025B.xlsx"  # <-- cambia aquí

# Exportar resultados de comparación a CSV
guardar_csv = True
salida_dir = "outputs_notebook"  # carpeta de salida para CSV


## Utilidades: normalización, lectura y tipado simple

In [38]:
def remove_accents(s: str) -> str:
    import unicodedata
    return "".join(ch for ch in unicodedata.normalize("NFKD", s) if not unicodedata.combining(ch))

def to_snake_case(s: str) -> str:
    s = re.sub(r"[^\w\s-]", " ", s, flags=re.UNICODE)
    s = re.sub(r"[-\s]+", "_", s.strip().lower())
    s = re.sub(r"_+", "_", s)
    return s

def normalize_headers(df: pd.DataFrame) -> pd.DataFrame:
    new_cols, seen = [], {}
    for c in df.columns:
        name = to_snake_case(remove_accents(str(c).strip())) or "columna"
        if name in seen:
            seen[name] += 1
            name = f"{name}_{seen[name]}"
        else:
            seen[name] = 0
        new_cols.append(name)
    out = df.copy()
    out.columns = new_cols
    return out

def read_excel_all_sheets(path: str, sheets_cfg: Union[str, List[str]] = "all") -> Dict[str, pd.DataFrame]:
    xls = pd.ExcelFile(path)  # requiere openpyxl
    if sheets_cfg == "all":
        sheets = xls.sheet_names
    elif isinstance(sheets_cfg, list):
        sheets = sheets_cfg
    else:
        raise ValueError("sheets_cfg debe ser 'all' o lista de nombres de hoja.")
    return {sh: pd.read_excel(path, sheet_name=sh, dtype="object") for sh in sheets}

def unify_sheets(frames: Dict[str, pd.DataFrame]) -> pd.DataFrame:
    frames_norm = {sh: normalize_headers(df) for sh, df in frames.items()}
    all_cols = set()
    for df in frames_norm.values():
        all_cols.update(df.columns.tolist())
    all_cols = list(all_cols)
    aligned = []
    for sh, df in frames_norm.items():
        tmp = df.copy()
        for c in all_cols:
            if c not in tmp.columns:
                tmp[c] = pd.NA
        tmp = tmp[all_cols]
        tmp["_sheet"] = sh
        aligned.append(tmp)
    return pd.concat(aligned, ignore_index=True)

def build_simple_dtypes(df: pd.DataFrame) -> Dict[str, str]:
    overrides = {}
    for c in df.columns:
        if c == "_sheet":
            overrides[c] = "string"; continue
        s = df[c]
        if s.dropna().empty:
            overrides[c] = "string"; continue
        nums = pd.to_numeric(s, errors="coerce")
        mask = s.notna()
        all_numeric = nums[mask].notna().all()
        if all_numeric:
            overrides[c] = "int" if (nums[mask] % 1 == 0).all() else "float"
        else:
            overrides[c] = "string"
    return overrides

def apply_dtypes(df: pd.DataFrame, dtypes: Dict[str, str]) -> pd.DataFrame:
    out = df.copy()
    for col, t in dtypes.items():
        if col not in out.columns: continue
        try:
            if t == "string":
                out[col] = out[col].astype("string")
            elif t == "int":
                out[col] = pd.to_numeric(out[col], errors="coerce").astype("Int64")
            elif t == "float":
                out[col] = pd.to_numeric(out[col], errors="coerce")
            else:
                out[col] = out[col].astype("string")
        except Exception:
            out[col] = out[col].astype("string")
    return out


## Preparar `master` con `row_index` como identificador

In [39]:
def prepare_master(excel_path: str) -> pd.DataFrame:
    frames = read_excel_all_sheets(excel_path, "all")
    df = unify_sheets(frames)
    inferred = build_simple_dtypes(df)
    df = apply_dtypes(df, inferred)
    # Asegurar índice limpio y crear 'row_index' como primera columna
    df = df.reset_index(drop=True)
    df.insert(0, "row_index", df.index.astype(int))
    return df


## Cargar Excel A → `master_a`

In [40]:
master_a = prepare_master(excel_a)
print(master_a.shape)
master_a.head(3)

c:\Users\andra\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


(671, 69)


,row_index,tipo_cargo,participacion_y_organizacion_de_colectivos_academicos_de_debate_o_intercambio_de_metodologias_y_experiencias_de_ensenanza,profesor,total_horas_actividades_de_docencia,participacion_como_revisor_de_libros_o_capitulos_revistas_indexadas_o_arbitrarias_o_de_publicaciones_revisada_por_pares,diseno_e_imparticion_de_cursos_de_educacion_continua_capacitacion_y_actualizacion_profesional_induccion_al_personal_academico_vinculado_al_curso_de_nivelacion_siempre_que_no_tengan_remuneracion_adicional,direccion_y_participacion_en_proyectos_sociales_artisticos_productivos_y_empresariales_de_vinculacion_con_la_sociedad_articulados_a_la_docencia_e_innovacion_educativa_aprobados_por_el_vips,la_prestacion_de_servicios_al_medio_externo_siempre_y_cuando_no_tengan_una_remuneracion_adicional_tales_como_analisis_de_laboratorio_especializado_peritaje_judicial_colaboracion_en_la_revista_tecnica_documental_para_las_instituciones_del_estado_y_trabajos_de_consultoria_institucional,total_horas_d_i_g_p_v,...,profesor_1,directores_de_programas_de_doctorado,vacaciones_2024_b,total_horas_vs_exigibles,preparacion_y_actualizacion_de_clases_seminarios_talleres_entre_otros,estado,director_de_centros_de_investigacion_desarrollo_tecnologico_innovacion_o_prestacion_de_servicios,dedicacion_1,tipo,_sheet
0,0,TITULAR,<NA>,IDENTIFICACIÓN: 1705009304/ NOMBRE: HERNANDEZ ...,326,20,<NA>,<NA>,<NA>,992,...,HERNANDEZ ALVAREZ MYRIAM BEATRIZ,<NA>,80,0,54,PREPLANIFICADA,<NA>,TC,TITULAR,Export
1,1,TITULAR,<NA>,IDENTIFICACIÓN: 1804029732/ NOMBRE: BENALCAZAR...,290,<NA>,<NA>,<NA>,<NA>,992,...,BENALCAZAR PALACIOS MARCO ENRIQUE,<NA>,80,0,40,PREPLANIFICADA,<NA>,TC,TITULAR,Export
2,2,TITULAR,<NA>,IDENTIFICACIÓN: 1716602048/ NOMBRE: TORRES OLM...,257,<NA>,<NA>,<NA>,<NA>,992,...,TORRES OLMEDO JENNY GABRIELA,<NA>,80,0,41,PREPLANIFICADA,<NA>,TC,TITULAR,Export


## Cargar Excel B → `master_b`

In [41]:
master_b = prepare_master(excel_b)
print(master_b.shape)
master_b.head(3)

c:\Users\andra\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


(660, 67)


,row_index,tipo_cargo,participacion_y_organizacion_de_colectivos_academicos_de_debate_o_intercambio_de_metodologias_y_experiencias_de_ensenanza,profesor,total_horas_actividades_de_docencia,participacion_como_revisor_de_libros_o_capitulos_revistas_indexadas_o_arbitrarias_o_de_publicaciones_revisada_por_pares,diseno_e_imparticion_de_cursos_de_educacion_continua_capacitacion_y_actualizacion_profesional_induccion_al_personal_academico_vinculado_al_curso_de_nivelacion_siempre_que_no_tengan_remuneracion_adicional,direccion_y_participacion_en_proyectos_sociales_artisticos_productivos_y_empresariales_de_vinculacion_con_la_sociedad_articulados_a_la_docencia_e_innovacion_educativa_aprobados_por_el_vips,la_prestacion_de_servicios_al_medio_externo_siempre_y_cuando_no_tengan_una_remuneracion_adicional_tales_como_analisis_de_laboratorio_especializado_peritaje_judicial_colaboracion_en_la_revista_tecnica_documental_para_las_instituciones_del_estado_y_trabajos_de_consultoria_institucional,total_horas_d_i_g_p_v,...,directores_de_programas_de_doctorado,coordinadores_de_carrera_y_de_programas_de_maestria,preparacion_elaboracion_aplicacion_y_calificacion_de_examenes_trabajos_y_practicas_consultas_academicas,profesor_1,preparacion_y_actualizacion_de_clases_seminarios_talleres_entre_otros,estado,director_de_centros_de_investigacion_desarrollo_tecnologico_innovacion_o_prestacion_de_servicios,dedicacion_1,tipo,_sheet
0,0,TITULAR,<NA>,IDENTIFICACIÓN: 0201669389/ NOMBRE: YACCHIREMA...,648.0,20,<NA>,<NA>,<NA>,976,...,<NA>,52,96,YACCHIREMA VARGAS DIANA CECILIA,96.0,ENVIADO INICIAL,<NA>,TC,TITULAR,Export
1,1,TITULAR,<NA>,IDENTIFICACIÓN: 0301435012/ NOMBRE: SAYAGO HER...,512.0,<NA>,<NA>,<NA>,<NA>,976,...,<NA>,316,72,SAYAGO HEREDIA JAIME PAUL,72.0,ENVIADO INICIAL,<NA>,TC,TITULAR,Export
2,2,TITULAR,<NA>,IDENTIFICACIÓN: 0703452169/ NOMBRE: VELEPUCHA ...,576.0,<NA>,<NA>,<NA>,<NA>,976,...,<NA>,<NA>,90,VELEPUCHA BONETT VICTOR VICENTE,90.0,ENVIADO INICIAL,<NA>,TC,TITULAR,Export


## Resolver columna a comparar (normalizando nombre)
El cuaderno busca la columna **normalizando** el nombre que escribiste en `columna_a_comparar`.

In [43]:
# === Comparación por IDENTIFICACIÓN: 'Cls-Dictadas' (B < A) ===
# 1) Resolver columnas en A y B
id_a = resolve_col(master_a, "Identificación")
id_b = resolve_col(master_b, "Identificación")
cls_a = resolve_col(master_a, "Cls-Dictadas")
cls_b = resolve_col(master_b, "Cls-Dictadas")

# 2) Funciones de limpieza
import unicodedata, re

def _remove_accents(s: str) -> str:
    return "".join(ch for ch in unicodedata.normalize("NFKD", s) if not unicodedata.combining(ch))

# normaliza IDs para comparar: str, trim, lowercase, sin acentos
def _norm_id_series(s):
    return (
        s.astype(str)
         .map(lambda v: "" if v is None else str(v))
         .str.strip()
         .map(lambda v: _remove_accents(v).lower())
    )

# 3) Preparar tablas base
A = master_a[[id_a, cls_a]].copy()
B = master_b[[id_b, cls_b]].copy()
A.columns = ["identificacion", "cls"]
B.columns = ["identificacion", "cls"]

# 4) Marcar e ignorar IDs inválidas (vacías o "sin identificacion")
INVALID_TOKENS = {"", "sin identificacion", "sin identificacion.", "s/i", "n/a", "na", "no aplica"}

A["identificacion_norm"] = _norm_id_series(A["identificacion"])
B["identificacion_norm"] = _norm_id_series(B["identificacion"])

invalid_A = A["identificacion_norm"].isin(INVALID_TOKENS)
invalid_B = B["identificacion_norm"].isin(INVALID_TOKENS)

cant_invalid_A = int(invalid_A.sum())
cant_invalid_B = int(invalid_B.sum())

# 5) Quedarse solo con válidas
A_valid = A[~invalid_A].copy()
B_valid = B[~invalid_B].copy()

# 6) Coerción a numérico y agregación por identificación (suma)
A_valid["cls_num"] = pd.to_numeric(A_valid["cls"], errors="coerce").fillna(0)
B_valid["cls_num"] = pd.to_numeric(B_valid["cls"], errors="coerce").fillna(0)

A_grp = (A_valid.groupby("identificacion_norm", as_index=True)["cls_num"]
         .sum().rename("cls_A").to_frame())
B_grp = (B_valid.groupby("identificacion_norm", as_index=True)["cls_num"]
         .sum().rename("cls_B").to_frame())

# 7) Unir por identificación y filtrar donde B < A
cmp_df = A_grp.join(B_grp, how="inner")
reporte_ids_b_menor = cmp_df.loc[cmp_df["cls_B"] < cmp_df["cls_A"]].copy()
reporte_ids_b_menor["diferencia_A_minus_B"] = reporte_ids_b_menor["cls_A"] - reporte_ids_b_menor["cls_B"]
reporte_ids_b_menor = reporte_ids_b_menor.reset_index().rename(columns={"identificacion_norm": "identificacion"})

# 8) Mostrar resumen y primeras filas
print({
    "ids_invalidas_en_A": cant_invalid_A,
    "ids_invalidas_en_B": cant_invalid_B,
    "total_ids_A_validas": int(len(A_grp)),
    "total_ids_B_validas": int(len(B_grp)),
    "ids_con_B_menor_que_A": int(len(reporte_ids_b_menor)),
})

reporte_ids_b_menor.head(50)


{'ids_invalidas_en_A': 9, 'ids_invalidas_en_B': 57, 'total_ids_A_validas': 641, 'total_ids_B_validas': 602, 'ids_con_B_menor_que_A': 240}


,identificacion,cls_A,cls_B,diferencia_A_minus_B
0,0101827806,276,252,24
1,0200590099,270,0,270
2,0301435012,360,216,144
3,0400588273,216,162,54
4,0401036272,360,0,360
5,0401110481,324,306,18
6,0401465695,360,306,54
7,0401771597,432,416,16
8,0401847900,360,342,18
9,0502343148,256,216,40
